In [1]:
import scripts.proj1_helpers as helper
import implementations as imp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import time
%matplotlib inline
%load_ext autoreload
%autoreload 2

## Feature engineering

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
dfb = pd.read_csv('test.csv')

In [ ]:
len(df[df == -999.0])

In [ ]:
for i, c in enumerate(df.columns):
    print(i, c)

In [ ]:
df.PRI_jet_num.unique()

## Import the data

boson is -1

not boson is 1

In [2]:
y_train, x_train, ids_train = helper.load_csv_data('train.csv')
y_test, x_test, ids_test = helper.load_csv_data('test.csv')
print(x_train.shape)
print(x_test.shape)

(250000, 30)
(568238, 30)


In [3]:
print('Number of boson:', np.count_nonzero(y_train-1))
print('Number of other:', np.count_nonzero(y_train+1))

Number of boson: 164333
Number of other: 85667


In [4]:
def col_standardize(x):
    means = [np.mean(col) for col in x.T]
    stds = [np.std(col) for col in x.T]
    return (x - means) / stds

In [5]:
def standardize(x):
    return (x - np.mean(x, axis=0)) / np.std(x, axis=0)

In [6]:
std_x_train = standardize(x_train)
std_x_test = standardize(x_test)

In [7]:
col_std_x_train = col_standardize(x_train)
col_std_x_test = col_standardize(x_test)

In [8]:
y_train[y_train < 0] = 0

In [11]:
def mean_spec(data):
    for column in data.T:
        temp = 0
        agg = 0
        for elem in column:
            if elem != -999.0:
                temp += elem
                agg += 1
        column[column == -999.0] = temp / agg 

## Logistic regression per category

In [9]:
x_test[:, 22]

array([ 0.,  1.,  0., ...,  0.,  1.,  0.])

In [10]:
x_test.shape, x_train.shape

((568238, 30), (250000, 30))

In [ ]:
cat_col = 22
PRI_jet_nums = np.unique(x_train[:, cat_col])
predictions = np.zeros(x_test.shape[0])

for num in PRI_jet_nums:
    cat_indices = np.where(x_train[:, cat_col] == num)
    
    x_train_cat = x_train[cat_indices]
    x_train_cat = np.delete(x_train_cat, cat_col, axis=1)
    
    stds = np.std(x_train_cat, axis=0)
    deleted_cols_ids = np.where(stds == 0)
    x_train_cat = np.delete(x_train_cat, deleted_cols_ids, axis=1)
    mean_spec(x_train_cat)
    x_train_cat = standardize(x_train_cat)
    
    _, w, _ = imp.logistic_regression(y_train[cat_indices],
                                      x_train_cat,
                                      max_iter=100,
                                      threshold=10**(-4))
    
    x_test_cat = x_test[cat_indices]
    x_test_cat = np.delete(x_test_cat, cat_col, axis=1)
    x_test_cat = np.delete(x_test_cat, deleted_cols_ids, axis=1)
    x_test_cat = standardize(x_test_cat)
    
    x_test_cat = np.hstack((np.ones((x_test_cat.shape[0], 1)), x_test_cat))
    
    predictions_cat = helper.predict_labels(w, x_test_cat)
    predictions[cat_indices] = predictions_cat

Current iteration=0, the loss=69254.41425128581
Current iteration=1, the loss=69208.90428414074
Current iteration=2, the loss=69163.48523946217
Current iteration=3, the loss=69072.8737758171
Current iteration=4, the loss=68937.5655872601
Current iteration=5, the loss=68668.96439718782
Current iteration=6, the loss=68183.57351498472
Current iteration=7, the loss=67323.85833983579
Current iteration=8, the loss=65847.37089690354
Current iteration=9, the loss=63394.21388079635
Current iteration=10, the loss=59637.81527346663
Current iteration=11, the loss=54506.90755529719
Current iteration=12, the loss=48659.72656265644
Current iteration=13, the loss=43429.53992474085
Current iteration=14, the loss=40125.34365060623
Current iteration=15, the loss=39170.31737022318
Current iteration=16, the loss=39100.76520710189
Current iteration=17, the loss=39100.21091886534


/Users/Alexis/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:30: DeprecationWarning: assignment will raise an error in the future, most likely because your index result shape does not match the value array shape. You can use `arr.flat[index] = values` to keep the old behaviour.


Current iteration=0, the loss=53749.4049693404
Current iteration=1, the loss=53729.664840458056
Current iteration=2, the loss=53709.96412509426
Current iteration=3, the loss=53690.302478978076
Current iteration=4, the loss=53670.68037978958
Current iteration=5, the loss=53631.5340480715
Current iteration=6, the loss=53573.07684778826
Current iteration=7, the loss=53495.6367103348
Current iteration=8, the loss=53380.51055058667
Current iteration=9, the loss=53210.11287990314
Current iteration=10, the loss=52968.794032836115
Current iteration=11, the loss=52643.855316932335
Current iteration=12, the loss=52192.383479287
Current iteration=13, the loss=51568.250995956434
Current iteration=14, the loss=50748.900943282744
Current iteration=15, the loss=49715.66181867434
Current iteration=16, the loss=48475.246658135584
Current iteration=17, the loss=47088.34100470683
Current iteration=18, the loss=45659.71090935227
Current iteration=19, the loss=44336.729726831414
Current iteration=20, the l

In [ ]:
helper.create_csv_submission(ids_test, predictions, 'separated.csv')

## Logistic regression

In [ ]:
niter = 5000

In [ ]:
start = time.perf_counter()
l, w, data = imp.stochastic_logistic_regression(y_train, std_x_train, max_iter=niter, batch_size=150, threshold=10**(-4))
end = time.perf_counter()

In [ ]:
print("Runtime:", end - start)

In [ ]:
len(data)

In [ ]:
plt.plot(data[100:])

In [ ]:
print("Loss first iteration", data[0])
print("Loss iteration at iteration", niter, data[-1])
print("delta of losses", data[0] - data[-1])

In [ ]:
std_x_test = np.hstack((np.ones((std_x_test.shape[0], 1)), std_x_test))                   

In [ ]:
predictions = helper.predict_labels(w, std_x_test)

In [ ]:
predictions

In [ ]:
helper.create_csv_submission(ids_test, predictions, 'newton.csv')

### Raw least squares (score: ?)

In [ ]:
(w, loss) = imp.least_squares(y_train, x_train)
loss

### Std least squares (score: 0.73)

In [ ]:
(w, loss) = imp.least_squares(y_train, col_std_x_train)
loss

In [ ]:
(w, loss) = imp.least_squares(y_train, std_x_train)
loss

### Eigenvalues reduction (score: 0.62)

In [ ]:
a = np.matrix([[1,2],[3,4]])
np.tile(a, 2)

In [ ]:
std_x_train = standardize(x_train)

In [ ]:
u, s, v = np.linalg.svd(std_x_train, full_matrices=False)
print('u shape:', u.shape)
print('s shape:', s.shape)
print('v shape:', v.shape)

In [ ]:
plt.plot(s)
plt.yscale('log')
plt.title('log')
plt.grid(True)
plt.show()

In [ ]:
shortened_x_train = u[:, :23] @ np.diag(s[:23]) @ v[:23,:]

In [ ]:
shortened_x_train.shape

In [ ]:
(w, loss) = imp.least_squares(y_train, x_train)

In [ ]:
loss

In [ ]:
y_pred = helper.predict_labels(w, x_test)

In [ ]:
print('Number of boson:', np.count_nonzero(y_pred+1))
print('Number of other:', np.count_nonzero(y_pred-1))

In [ ]:
helper.create_csv_submission(ids_test, y_pred, 'shortened_eigenvalues_submission.csv')

In [ ]:
unique_values_per_column_count = [len(set(col)) for col in train_data.T]

In [ ]:
unique_values_per_column_count

In [ ]:
def y_map(y):
    if y == -1:
        return 0
    else:
        return 0.2

y_train_mapped = np.vectorize(y_map)(y_train)

### PCA

In [ ]:
square_train_data = std_x_train.T @ std_x_train

In [ ]:
w, v = np.linalg.eigh(square_train_data)

In [ ]:
print(w.shape[0])
print(v.shape)

In [ ]:
w

In [ ]:
def keep_variance(percentage, vec):
    r = list(range(1, w.shape[0] + 1))
    total = np.sum(vec)
    sums = list(map(lambda i: np.sum(vec[-i:]), r))
    ratio = sums / total
    return np.argmin(abs(ratio - percentage)) + 1

In [ ]:
index_keeper = keep_variance(0.9, w)
print(index_keeper)

In [ ]:
plt.plot(w)
plt.yscale('log')
plt.title('log')
plt.grid(True)
plt.show()

In [ ]:
filtered_v = v[:,-index_keeper:]

In [ ]:
filtered_v.shape

In [ ]:
project_x_train = std_x_train @ filtered_v

In [ ]:
tupled_boson = np.array(list(zip(*filter(lambda pair: pair[1] == -1, zip(project_x_train.tolist(), y_train))))[0])
tupled_other = np.array(list(zip(*filter(lambda pair: pair[1] == 1, zip(project_x_train.tolist(), y_train))))[0])

In [ ]:
tupled_boson.shape

In [ ]:
plt.plot(tupled_boson[1000:2000, :1], tupled_boson[1000:2000, 1:], 'bo')
plt.plot(tupled_other[1000:2000, :1], tupled_other[1000:2000, 1:], 'ro')
plt.show()

In [ ]:
(weight, loss) = imp.least_squares(y_train, project_x_train)

In [ ]:
loss

In [ ]:
y_pred = helper.predict_labels(weight, std_x_test @ filtered_v)

In [ ]:
print('Number of boson:', np.count_nonzero(y_pred+1))
print('Number of other:', np.count_nonzero(y_pred-1))

### Polynomial feature

In [ ]:
np.array(range(2))

In [ ]:
def polynomial_enhancement(x, deg):
    stacked_x = np.tile(x, deg+1)
    power_vec = np.repeat(np.array(range(deg+1)), x.shape[1])
    return stacked_x ** power_vec

In [ ]:
enhanced_col_std_x_train = polynomial_enhancement(std_x_train, 9)
enhanced_col_std_x_test = polynomial_enhancement(std_x_test, 9)
enhanced_col_std_x_train.shape

In [ ]:
def pseudo_least_squares(y, x):
    U, S, V = np.linalg.svd(x, full_matrices=False)
    w = V.T @ np.diag(1/S) @ U.T @ y
    loss = imp.mse(y, x, w)
    return (w, loss)

In [ ]:
(w, loss) = pseudo_least_squares(y_train, enhanced_col_std_x_train)
loss

In [ ]:
y_pred = helper.predict_labels(w, enhanced_col_std_x_test)

In [ ]:
helper.create_csv_submission(ids_test, y_pred, 'basic_poly_enhancement_9.csv')

#### Cross validation

In [ ]:
acc, loss_train, loss_test, w = imp.cross_validation_v2(y_train, std_x_train, 10, imp.pseudo_least_squares, 5)

In [ ]:
acc

In [ ]:
enhanced_col_std_x_test = imp.polynomial_enhancement(std_x_test, 5)

In [ ]:
y_pred = helper.predict_labels(w, enhanced_col_std_x_test)

In [ ]:
helper.create_csv_submission(ids_test, y_pred, '0_8_accuracy_poly_5.csv')

In [ ]:
test = []

test.append(1)
test

In [ ]:
acc, loss_train, loss_test, w = imp.cross_validation_v2(y_train, std_x_train, 10, imp.pseudo_least_squares, 5)